In [132]:
import requests
GECKO_TERMINAL = "https://app.geckoterminal.com/api/p1/pools"

def page_count():
    x = requests.get(GECKO_TERMINAL).json()
    return x['links']['last']['meta']['series'][-1]


def page_result(page_number):
    return requests.get(
        GECKO_TERMINAL,
        {"include": "dex,dex.network,pool_metric,tokens",
         "page": page_number,
            # "include_network_metrics": True
            }
    ).json()


r = []
# for page_number in range(1, page_count()):
#     try:
#         r.append(page_result(page_number))
#     except Exception as e:
#         print(e)

for page_number in range(1, 2):
    r.append(page_result(page_number))




In [1]:
import json 
with open("gecko_terminal_pools.json") as f: 
    r = json.load(f)

In [2]:
len(r)

80

In [3]:
dexs = {}
tokens = {}
network = {}
pool_metric = {}
pools = {}
dex_metrics ={}
rel_types = ["dex", "token","network","pool_metric",  "pool"]

results = {
    "pool":pools,
    "dex": dexs,
    "token": tokens,
    "network": network,
    "pool_metric": pool_metric,
    "dex_metric":dex_metrics,
}
count = 0
_types = set()
for result in r:
    for element in [*result['data'], *result['included']]:
        count +=1
        _types.add(element['type'])
        match element['type']:
            case "dex":
                dexs[element['id']] = element
            case "dex_metric":
                dex_metrics[element['id']] = element
            case "pool":
                pools[element['id']] = element
            case "token":
                tokens[element['id']] = element
            case "network":
                network[element['id']] = element
            case "pool_metric":
                pool_metric[element['id']] = element
_types , count

({'dex', 'network', 'pool', 'pool_metric', 'token'}, 31215)

In [18]:
for _type in _types:
    type_result=results[_type]
    for type_id , type in type_result.items():
        if 'relationships' not in type:
            continue
        for rel_type, rel in type['relationships'].items():
            if rel_type in ['network_metric','dex_metric']:
                continue
            rel_data = rel['data']
            if isinstance(rel_data, list):
                for i, _elm in enumerate(rel_data):
                    _elem_id = _elm['id']
                    _elem_type = _elm['type']
                    if rel_type not in type_result[type_id] or isinstance(type_result[type_id].get(rel_type), list):
                        type_result[type_id][rel_type] = {}
                        
                    type_result[type_id][rel_type][_elem_id] = results[_elem_type][_elem_id]
            else:    
                if rel_data['id'] not in results[rel_type]:
                    print(rel_data['id'], rel_type)
                else:
                    type_result[type_id][rel_type] = results[rel_type][rel_data['id']]
        for rel_type in rel_types:
            if rel_type not in type_result[type_id]:
                type_result[type_id][rel_type] = None
{k:len(v) for k,v in results.items()}


{'pool': 7078,
 'dex': 270,
 'token': 4685,
 'network': 59,
 'pool_metric': 7078,
 'dex_metric': 0}

{'pool': 7078,
 'dex': 270,
 'token': 4685,
 'network': 59,
 'pool_metric': 7078,
 'dex_metric': 0}

In [5]:
import json 
with open("tmp.json","w+") as f: json.dump(list(pools.values())[0],f)

In [7]:
from pydantic import BaseModel
from typing import Optional, List, Set
import logging

logger = logging.getLogger(__name__)


class Token(BaseModel):
    chainId: int
    address: str
    name: str
    symbol: str
    decimals: int
    priceUSD: Optional[str]  # Later convert to float
    tags: Optional[List[str]]
    coingeckoId: Optional[str]
    lifiId: Optional[str]  # ! In lifi list is coinkey
    listedIn: Optional[List[str]]
    logoURI: Optional[str]
    verify: bool = False

class Pair(Token):
    dex: str
    tokens: List[Token]
    totalSupply: Optional[int]

In [ ]:
from web3 import Web3
valid_tokens = {}

for i, (token_id,token) in enumerate(tokens.items()):
    try:
        valid_tokens[token['id']] = Token(
            chainId=token['network']['attributes']['chain_id'],
            address=Web3.toChecksumAddress(token['attributes']['address']),
            symbol=token['attributes']['symbol'],
            name=token['attributes']['name'],
            decimals=-1,
            logoURI=token['attributes']['image_url'],
        )
    except ValueError as e:
        # print(token_id, token['network']['attributes']['chain_id'], e)
        pass
    except Exception as e:
        print(i)
        raise e

print(f"\n\n{len(valid_tokens)=}  errors {len(tokens)- len(valid_tokens)=}")

In [29]:
valid_pools = {}

for i, (pool_id,pool) in enumerate(pools.items()):
    try:
        pool_tokens = [
         valid_tokens[_] for _ in pool['tokens']
            ]
         
        valid_pools[pool['id']] = Pair(
            chainId=pool['dex']['network']['attributes']['chain_id'],
            address=Web3.toChecksumAddress(pool['attributes']['address']),
            symbol=pool['dex']['attributes']['identifier'] + "LP (" + pool['attributes']['name'] + ")",
            name=pool['attributes']['name'],
            decimals=-1,
            logoURI=pool['dex']['attributes']['image_url'],
            dex=pool['dex']['attributes']['identifier'],
            tokens=pool_tokens
        )
    except KeyError as e:
        print(pool_id, pool['dex']['network']['attributes']['chain_id'], e)
        pass
    except ValueError as e:
        pass
    except Exception as e:
        print(i)
        raise e

148390732 None '2984234'
148390726 None '2984234'
148914778 None '4045901'
148914793 None '4046257'
155124253 None '4053460'
148914862 None '4046026'
148914935 None '4047758'
148914884 None '4046258'
148914840 None '4046366'
148390826 None '2984234'
148914915 None '4046026'
148916424 None '4048260'
148915027 None '4052264'
148914906 None '4046257'
148917519 None '4047549'
148914861 None '4046924'
148915026 None '4047769'
148914920 None '4046258'
148914780 None '4045901'
148914779 None '4045999'
148595407 None '2989037'
148915235 None '4052264'
148390764 None '2984309'
148390788 None '2984338'
148915020 None '4055330'
148390803 None '2984369'
148390772 None '2984322'
149411800 None '5050866'
148595389 None '2989037'
148914881 None '4046661'
148914875 None '4047293'
148915054 None '4045828'
148914926 None '4046026'
148915199 None '4055934'
148391207 None '2984234'
148915408 None '4055396'
148914958 None '4050047'
148915208 None '4047912'
148922537 None '4046020'
148395286 None '2984322'


In [25]:
len(valid_pools)

6959

In [27]:
import functools
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, BaseModel):
            return obj.dict()
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)


dump = functools.partial(json.dump, cls=SetEncoder)


In [31]:
with open("raw.json","w+") as f:
    dump(valid_pools,f)

In [35]:
all_pairs = [_ for _ in valid_pools.values()]
with open("all_pairs.json","w+") as f:
    dump(all_pairs,f)
len(all_pairs)

6959

In [36]:
chain_seperated ={}
count = 0
for pair in valid_pools.values():
    pair: Pair
    if pair.chainId not in chain_seperated:
        chain_seperated[pair.chainId] = []
    chain_seperated[pair.chainId] = pair
    count += 1

with open("pairs_chain_seperated.json","w+") as f:
    dump(chain_seperated, f)
count 

6959